In [14]:
library(ggplot2)
library(microbiome)
library(phyloseq)
library("tidyverse")
library("dplyr")
library("randomForestSRC")
library("ggpmisc")
library(patchwork)
library("speedyseq")
library(microViz)

In [2]:
packageVersion('randomForestSRC')

[1] ‘3.2.2’

In [3]:
load("/Users/stephanie.rosales/Documents/omics_projects/POR_16S/CoralPersistence/data/POR_counts_4_analysis.RData")
load("/Users/stephanie.rosales/Documents/omics_projects/POR_16S/CoralPersistence/data/POR_taxo_4_analysis.RData")


In [4]:
taxo_mat <- POR_taxo_4_analysis %>%
  rownames_to_column %>%
  filter(rowname != 'ASV_2') %>%
column_to_rownames("rowname")  %>%
mutate(family = ifelse(row.names(.) == "ASV_94", "ASV_94:Bacteroidia", family)) %>%
#mutate(family = ifelse(row.names(.) == "ASV_68", "ASV_68", family)) %>%
  mutate(domain = ifelse(row.names(.) == "ASV_5", "ASV_5:core", domain)) %>%
  mutate(phylum = ifelse(row.names(.) == "ASV_5", "ASV_5:core", phylum)) %>%
  mutate(class = ifelse(row.names(.) == "ASV_5", "ASV_5:core", class)) %>%
   mutate(order = ifelse(row.names(.) == "ASV_5", "ASV_5:core", order)) %>%
  mutate(family = ifelse(row.names(.) == "ASV_5", "ASV_5:core", family)) %>%
mutate(genus = ifelse(row.names(.) == "ASV_5", "ASV_5:core", genus)) %>%
mutate(species = ifelse(row.names(.) == "ASV_5", "ASV_5:core", species)) %>%
 mutate(domain = ifelse(row.names(.) == "ASV_6", "ASV_6:core", domain)) %>%
  mutate(phylum = ifelse(row.names(.) == "ASV_6", "ASV_6:core", phylum)) %>%
  mutate(class = ifelse(row.names(.) == "ASV_6", "ASV_6:core", class)) %>%
   mutate(order = ifelse(row.names(.) == "ASV_6", "ASV_6:core", order)) %>%
  mutate(family = ifelse(row.names(.) == "ASV_6", "ASV_6:core", family)) %>%
mutate(genus = ifelse(row.names(.) == "ASV_6", "ASV_6:core", genus)) %>%
mutate(species = ifelse(row.names(.) == "ASV_6", "ASV_6:core", species)) %>%
mutate(class = ifelse(row.names(.) == "ASV_7", "SAR324:core", class)) %>%
   mutate(order = ifelse(row.names(.) == "ASV_7", "SAR324:core", order)) %>%
  mutate(family = ifelse(row.names(.) == "ASV_7", "SAR324:core", family)) %>%
mutate(genus = ifelse(row.names(.) == "ASV_7", "SAR324:core", genus)) %>%
mutate(species = ifelse(row.names(.) == "ASV_7", "SAR324:core", species)) %>%
mutate(genus = ifelse(row.names(.) == "ASV_23", "Cyanobiaceae:core", genus)) %>%
mutate(species = ifelse(row.names(.) == "ASV_23", "Cyanobiaceae:core", species)) 

In [5]:
meta = read.csv ("/Users/stephanie.rosales/Documents/omics_projects/POR_16S/CoralPersistence/metadata/POR_combo_meta.csv",
row.names=1, header = T)%>%
  mutate(
    Month = factor(Month, levels = month.name)
  ) %>%
  arrange(Month) %>%
arrange(collection_date)

ASV <- otu_table(as.matrix(POR_counts_4_analysis), 
                 taxa_are_rows = T)
TAX <- tax_table(as.matrix(taxo_mat))
SAMP <- sample_data(meta)

ps <- phyloseq(ASV, TAX, SAMP)
ps

phyloseq-class experiment-level object
otu_table()   OTU Table:          [ 27533 taxa and 363 samples ]:
sample_data() Sample Data:        [ 363 samples by 58 sample variables ]:
tax_table()   Taxonomy Table:     [ 27533 taxa by 7 taxonomic ranks ]:
taxa are rows

In [6]:
euk_asvs <- c("ASV_10673","ASV_10883",	"ASV_11806","ASV_12333",	"ASV_12335",	"ASV_13587",	"ASV_14757",	"ASV_15238",	"ASV_15259",	"ASV_16364",	"ASV_16936",	"ASV_17406",	"ASV_18096",	"ASV_18375",	"ASV_18682",	"ASV_19529",	"ASV_19795",	"ASV_19953",	"ASV_2",	"ASV_20640",	"ASV_2100",	"ASV_22284",	"ASV_22692",	"ASV_23038",	"ASV_23144",	"ASV_24135",	"ASV_24243",	"ASV_27014",	"ASV_29665",	"ASV_30547",	"ASV_30626",	"ASV_30884",	"ASV_4161",	"ASV_4498",	"ASV_476",	"ASV_4797",	"ASV_4966",	"ASV_5018",	"ASV_5118",	"ASV_5154",	"ASV_6282",	"ASV_653",	"ASV_6843",	"ASV_7264",	"ASV_7427",	"ASV_8",	"ASV_9248",	"ASV_9692")
allTaxa = taxa_names(ps)
allTaxa <- allTaxa[!(allTaxa %in% euk_asvs)]
ps_fill_euk = prune_taxa(allTaxa, ps)
# new phyloseq object with just the taxa you kept.
ps_fill_euk

phyloseq-class experiment-level object
otu_table()   OTU Table:          [ 27486 taxa and 363 samples ]:
sample_data() Sample Data:        [ 363 samples by 58 sample variables ]:
tax_table()   Taxonomy Table:     [ 27486 taxa by 7 taxonomic ranks ]:
taxa are rows

In [7]:
ps_merge = merge_samples2(ps_fill_euk, "geno_clus")
ps_merge

phyloseq-class experiment-level object
otu_table()   OTU Table:          [ 27486 taxa and 125 samples ]:
sample_data() Sample Data:        [ 125 samples by 58 sample variables ]:
tax_table()   Taxonomy Table:     [ 27486 taxa by 7 taxonomic ranks ]:
taxa are rows

In [8]:
ps_fill = filter_taxa(ps_merge, function(x) sum(x > 5) > (0.1*length(x)), TRUE)  %>% tax_fix()
ps_fill 
ps_fill

Row named: ASV_10
contains no non-unknown values, returning:
'ASV_10' for all replaced levels.
Consider editing this tax_table entry manually.

Row named: ASV_13
contains no non-unknown values, returning:
'ASV_13' for all replaced levels.
Consider editing this tax_table entry manually.

Row named: ASV_21
contains no non-unknown values, returning:
'ASV_21' for all replaced levels.
Consider editing this tax_table entry manually.

Row named: ASV_25
contains no non-unknown values, returning:
'ASV_25' for all replaced levels.
Consider editing this tax_table entry manually.

Row named: ASV_68
contains no non-unknown values, returning:
'ASV_68' for all replaced levels.
Consider editing this tax_table entry manually.

Row named: ASV_81
contains no non-unknown values, returning:
'ASV_81' for all replaced levels.
Consider editing this tax_table entry manually.

Row named: ASV_88
contains no non-unknown values, returning:
'ASV_88' for all replaced levels.
Consider editing this tax_table entry man

phyloseq-class experiment-level object
otu_table()   OTU Table:          [ 1166 taxa and 125 samples ]:
sample_data() Sample Data:        [ 125 samples by 58 sample variables ]:
tax_table()   Taxonomy Table:     [ 1166 taxa by 7 taxonomic ranks ]:
taxa are rows

phyloseq-class experiment-level object
otu_table()   OTU Table:          [ 1166 taxa and 125 samples ]:
sample_data() Sample Data:        [ 125 samples by 58 sample variables ]:
tax_table()   Taxonomy Table:     [ 1166 taxa by 7 taxonomic ranks ]:
taxa are rows

In [9]:
#set seed
set.seed(2)

In [10]:
predictors <- t(otu_table(ps_fill))

# Make one column for response variable 
response <- as.factor(sample_data(ps_fill)$Samp_trip)

# Combine them into data frame
rf.data <- data.frame(response, predictors)
head(rf.data, n=2)


trip.classify <- rfsrc(response~., data = rf.data, ntree = 400,
                     importance="permute", csv.num=TRUE)
trip.classify

tax_trip=cbind(as.data.frame(trip.classify$importance), as.data.frame(tax_table(ps_fill))) %>%
arrange(-all) #%>%
#head(n=10)

head(tax_trip)




,response,ASV_479,ASV_893,ASV_365,ASV_535,ASV_2096,ASV_772,ASV_1957,ASV_1450,ASV_2272,⋯,ASV_5161,ASV_5172,ASV_5315,ASV_5568,ASV_6632,ASV_13462,ASV_13967,ASV_20040,ASV_20192,ASV_20228
,<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
ML2_78_cf_Trip_3,Trip_3,0,0,31,47,0,0,0,10,0,⋯,0,0,0,0,0,0,0,0,0,0
CN4_35_cf_Trip_3,Trip_3,0,10,0,133,0,9,22,216,0,⋯,4,0,0,0,0,0,0,0,0,0


                         Sample size: 125
           Frequency of class labels: 34, 33, 28, 30
                     Number of trees: 400
           Forest terminal node size: 1
       Average no. of terminal nodes: 20.2875
No. of variables tried at each split: 35
              Total no. of variables: 1166
       Resampling used to grow trees: swor
    Resample size used to grow trees: 79
                            Analysis: RF-C
                              Family: class
                      Splitting rule: gini *random*
       Number of random split points: 10
                   (OOB) Brier score: 0.08443749
        (OOB) Normalized Brier score: 0.45033328
                           (OOB) AUC: 0.99573625
   (OOB) Requested performance error: 0.024, 0.02941176, 0, 0.03571429, 0.03333333

Confusion matrix:

          predicted
  observed Trip_2 Trip_3 Trip_4 Trip_5 class.error
    Trip_2     33      1      0      0      0.0294
    Trip_3      0     33      0      0      0.0000
    Tr

,all,Trip_2,Trip_3,Trip_4,Trip_5,domain,phylum,class,order,family,genus,species
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
ASV_865,0.014092489,0.01598989,0.01647444,9.708149e-02,0.031713288,Bacteria,Proteobacteria,Gammaproteobacteria,Alteromonadales,Colwelliaceae,Thalassotalea,Thalassotalea genus
ASV_963,0.011912346,0.03797600,0.01853374,-7.544748e-18,0.070222281,Bacteria,Dadabacteria,Dadabacteriia,Dadabacteriales,Dadabacteriales order,Dadabacteriales order,Dadabacteriales order
ASV_151,0.011725300,0.00599621,0.02471165,2.184334e-02,0.077017985,Bacteria,Proteobacteria,Gammaproteobacteria,Pseudomonadales,Pseudomonadaceae,Pseudomonas,Pseudomonas genus
ASV_412,0.010674719,0.02198610,0.03294887,4.854075e-03,0.054365637,Bacteria,Bacteroidota,Bacteroidia,Flavobacteriales,Flavobacteriaceae,NS4 marine group,NS4 marine group genus
ASV_1299,0.010271716,0.02198610,0.01029652,2.184334e-02,0.058896106,Bacteria,Marinimicrobia (SAR406 clade),Marinimicrobia (SAR406 clade) phylum,Marinimicrobia (SAR406 clade) phylum,Marinimicrobia (SAR406 clade) phylum,Marinimicrobia (SAR406 clade) phylum,Marinimicrobia (SAR406 clade) phylum
ASV_1069,0.009468163,0.01798863,0.05148261,2.184334e-02,0.009060939,Bacteria,Proteobacteria,Alphaproteobacteria,SAR11 clade,Clade III,Clade III family,Clade III family


In [11]:
##Combine top important ASVs from each sampling trip

# Create a list to store the results
result_list <- list()

# Define the column names for sorting
sort_columns <- c("Trip_2", "Trip_3", "Trip_4", "Trip_5")

# Loop through the sorting columns
for (col in sort_columns) {
  result <- cbind(as.data.frame(trip.classify$importance), as.data.frame(tax_table(ps_fill))) %>%
    arrange(desc(get(col))) %>%
    head() %>%
    distinct()  # Remove duplicate rows
    
  # Assign the result to the list
  result_list[[paste0(col, "_5")]] <- result
}

# Combine the results into a single data frame
rf_trip_df <- do.call(rbind, result_list)


#write.csv(rf_trip_df, "/Users/stephanie.rosales/Documents/omics_projects/POR_16S/CoralPersistence/metadata/rf_trip_df.csv")

In [12]:
predictors <- t(otu_table(ps_fill))
dim(predictors)
# Make one column for our outcome/response variable 

response <- as.factor(sample_data(ps_fill)$reef)

# Combine them into 1 data frame
rf.data <- data.frame(response, predictors)
head(rf.data, n=2)


reef.classify <- rfsrc(response~., data = rf.data, ntree = 400,
                     importance="permute", csv.num=TRUE)
reef.classify

tax_reef= cbind(as.data.frame(reef.classify$importance), as.data.frame(tax_table(ps_fill))) %>%
arrange(-all) #%>%
#head()

tax_reef %>%
arrange(-ndr) %>%
head()



[1]  125 1166

,response,ASV_479,ASV_893,ASV_365,ASV_535,ASV_2096,ASV_772,ASV_1957,ASV_1450,ASV_2272,⋯,ASV_5161,ASV_5172,ASV_5315,ASV_5568,ASV_6632,ASV_13462,ASV_13967,ASV_20040,ASV_20192,ASV_20228
,<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
ML2_78_cf_Trip_3,cf,0,0,31,47,0,0,0,10,0,⋯,0,0,0,0,0,0,0,0,0,0
CN4_35_cf_Trip_3,cf,0,10,0,133,0,9,22,216,0,⋯,4,0,0,0,0,0,0,0,0,0


                         Sample size: 125
           Frequency of class labels: 31, 47, 47
                     Number of trees: 400
           Forest terminal node size: 1
       Average no. of terminal nodes: 19.665
No. of variables tried at each split: 35
              Total no. of variables: 1166
       Resampling used to grow trees: swor
    Resample size used to grow trees: 79
                            Analysis: RF-C
                              Family: class
                      Splitting rule: gini *random*
       Number of random split points: 10
                   (OOB) Brier score: 0.14105482
        (OOB) Normalized Brier score: 0.63474669
                           (OOB) AUC: 0.88550626
   (OOB) Requested performance error: 0.288, 0.77419355, 0.12765957, 0.12765957

Confusion matrix:

          predicted
  observed cf ndr pr class.error
       cf   7  13 11      0.7742
       ndr  4  41  2      0.1277
       pr   2   4 41      0.1277

      (OOB) Misclassification rate

,all,cf,ndr,pr,domain,phylum,class,order,family,genus,species
,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
ASV_109,0.010093047,0.015345139,0.04337684,0.018796630,Bacteria,Proteobacteria,Alphaproteobacteria,Rhodobacterales,Rhodobacteraceae,Rhodobacteraceae family,Rhodobacteraceae family
ASV_273,0.007253705,-0.008768651,0.03180968,0.026026103,Bacteria,Bacteroidota,Bacteroidia,Flavobacteriales,Flavobacteriaceae,Flavobacteriaceae family,Flavobacteriaceae family
ASV_252,0.006487859,0.010960814,0.03036379,0.008675368,Bacteria,Bacteria domain,Bacteria domain,Bacteria domain,Bacteria domain,Bacteria domain,Bacteria domain
ASV_443,0.005043731,-0.010960814,0.02602610,0.017350735,Bacteria,Bacteroidota,Bacteroidia,Flavobacteriales,NS9 marine group,NS9 marine group family,NS9 marine group family
ASV_14,0.004429137,0.015345139,0.02313431,-0.001445895,Bacteria,Proteobacteria,Gammaproteobacteria,Vibrionales,Vibrionaceae,Vibrionaceae family,Vibrionaceae family
ASV_46,0.005255499,-0.008768651,0.02313431,0.020242524,Bacteria,Cyanobacteria,Cyanobacteriia,Synechococcales,Cyanobiaceae,Synechococcus CC9902,Synechococcus CC9902 genus


In [13]:
##Combine top important ASVs from each reef
set.seed(2)
# Create a list to store the results
result_list <- list()

# Define the column names for sorting
sort_columns <- c("pr", "ndr", "cf")

# Loop through the sorting columns
for (col in sort_columns) {
  result <- cbind(as.data.frame(reef.classify$importance), as.data.frame(tax_table(ps_fill))) %>%
    arrange(desc(get(col))) %>%
    head() %>%
    distinct()  # Remove duplicate rows
    
  # Assign the result to the list
  result_list[[paste0(col, "_5")]] <- result
}

# Combine the results into a single data frame
rf_reef_df <- do.call(rbind, result_list)


#write.csv(rf_reef_df, "/Users/stephanie.rosales/Documents/omics_projects/POR_16S/CoralPersistence/metadata/rf_reef_df.csv",
 #        )